## Safe Intersection

### Environment

In [ ]:
SX, SY = grid.domain.hi[0] - grid.domain.lo[0], grid.domain.hi[1] - grid.domain.lo[1]

C30 = 0.6
C50 = 1.0

print('Creating intersection')
intersection = shp.intersection(shp.hyperplane(grid, normal=[-1, 1], offset=[SX * 4.60/8, SY * 2.67/6]),
                                shp.hyperplane(grid, normal=[1, 1],  offset=[SX * 6.40/8, SY * 2.67/6]),
                                shp.rectangle(grid, 
                                              target_min=[SX * 4.20/8, SY * 1.20/6], 
                                              target_max=[SX * 6.87/8, SY * 3.04/6]))
print('Creating kyrkogatan_right')
kyrkogatan_right = shp.union(intersection,
                             shp.rectangle(grid, 
                                           target_min=[SX * 0.00/8, SY * 1.20/6, -np.pi/5],
                                           target_max=[SX * 5.33/8, SY * 1.84/6, +np.pi/5]),
                             shp.rectangle(grid, 
                                           target_min=[SX * 6.00/8, SY * 1.20/6, -np.pi/5],
                                           target_max=[SX * 8.00/8, SY * 1.73/6, +np.pi/5]))
kyrkogatan_right = shp.intersection(kyrkogatan_right,
                                    implies(shp.upper_half_space(grid, X, SX * 3.00/8), 
                                            shp.rectangle(grid, axes=[V], target_min=[0.3], target_max=[C30])),
                                    implies(shp.lower_half_space(grid, X, SX * 3.01/8), 
                                            shp.rectangle(grid, axes=[V], target_min=[0.4], target_max=[C50])))
print('Creating kyrkogatan_left')
kyrkogatan_left = shp.union(intersection,
                            shp.rectangle(grid, 
                                          target_min=[SX * 0.00/8, SY * 1.85/6, +np.pi - np.pi/5], 
                                          target_max=[SX * 5.33/8, SY * 2.37/6, -np.pi + np.pi/5]),
                            shp.rectangle(grid, 
                                          target_min=[SX * 6.00/8, SY * 1.70/6, +np.pi - np.pi/5], 
                                          target_max=[SX * 8.00/8, SY * 2.24/6, -np.pi + np.pi/5]))
kyrkogatan_left = shp.intersection(kyrkogatan_left,
                                   implies(shp.upper_half_space(grid, X, SX * 3.00/8), 
                                           shp.rectangle(grid, axes=[V], target_min=[0.3], target_max=[C30])),
                                   implies(shp.lower_half_space(grid, X, SX * 3.01/8), 
                                           shp.rectangle(grid, axes=[V], target_min=[0.4], target_max=[C50])))
print('Creating nygatan_up')
nygatan_up = shp.union(intersection,
                       shp.rectangle(grid, 
                                     target_min=[SX * 5.60/8, SY * 2.27/6, +np.pi/2 - np.pi/5], 
                                     target_max=[SX * 6.14/8, SY * 6.00/6, +np.pi/2 + np.pi/5]),
                       shp.rectangle(grid, 
                                     target_min=[SX * 5.33/8, SY * 0.00/6, +np.pi/2 - np.pi/5], 
                                     target_max=[SX * 5.74/8, SY * 1.60/6, +np.pi/2 + np.pi/5]))
nygatan_up = shp.intersection(nygatan_up,
                              implies(shp.upper_half_space(grid, Y, SY * 4.00/6), 
                                      shp.rectangle(grid, axes=[V], target_min=[0.4], target_max=[C50])),
                              implies(shp.lower_half_space(grid, Y, SY * 4.01/6), 
                                      shp.rectangle(grid, axes=[V], target_min=[0.3], target_max=[C30])))
print('Creating nygatan_down')
nygatan_down = shp.union(intersection,
                         shp.rectangle(grid, 
                                       target_min=[SX * 4.94/8, SY * 2.26/6, -np.pi/2 - np.pi/5],
                                       target_max=[SX * 5.53/8, SY * 6.00/6, -np.pi/2 + np.pi/5]),
                         shp.rectangle(grid, 
                                       target_min=[SX * 5.07/8, SY * 0.00/6, -np.pi/2 - np.pi/5],
                                       target_max=[SX * 5.47/8, SY * 1.60/6, -np.pi/2 + np.pi/5]))
print('Creating parking_spots')
parking_spot_entry_1 = shp.rectangle(grid, 
                                     target_min=[SX * 2.00/8, SY * 5.33/6], 
                                     target_max=[SX * 2.53/8, SY * 5.73/6])
parking_spot_1 = shp.rectangle(grid, 
                               target_min=[SX * 2.13/8, SY * 5.54/6, +np.pi/2 - np.pi/5], 
                               target_max=[SX * 2.40/8, SY * 6.00/6, +np.pi/2 + np.pi/5])
parking_spot_entry_2 = shp.rectangle(grid, 
                                     target_min=[SX * 3.02/8, SY * 4.67/6], 
                                     target_max=[SX * 3.61/8, SY * 5.07/6])
parking_spot_2 = shp.rectangle(grid, 
                               target_min=[SX * 3.15/8, SY * 4.33/6, -np.pi/2 - np.pi/5], 
                               target_max=[SX * 3.47/8, SY * 4.80/6, -np.pi/2 + np.pi/5])
print('Creating parking_lot')
parking_lot = shp.union(shp.rectangle(grid, target_min=[SX * 1.46/8, SY * 2.73/6], target_max=[SX * 2.00/8, SY * 6.00/6]),  # left side
                        shp.rectangle(grid, target_min=[SX * 1.46/8, SY * 3.27/6], target_max=[SX * 4.40/8, SY * 3.74/6]),  # bottom
                        shp.rectangle(grid, target_min=[SX * 4.07/8, SY * 3.33/6], target_max=[SX * 4.60/8, SY * 5.47/6]),  # right side
                        shp.rectangle(grid, target_min=[SX * 1.47/8, SY * 4.87/6], target_max=[SX * 4.60/8, SY * 5.47/6]),  # top
                        shp.rectangle(grid, target_min=[SX * 1.47/8, SY * 2.13/6], target_max=[SX * 1.93/8, SY * 3.00/6]))  # entry
# parking_lot = shp.union(parking_lot,
#                         parking_spot_entry_1, parking_spot_1,
#                         parking_spot_entry_2, parking_spot_2)
parking_lot = shp.intersection(parking_lot,
                               shp.lower_half_space(grid, V, C30))
print('Creating entry_zone and exit_zone')
# exit_zone = shp.rectangle(grid, target_min=[SX * 5.67/8, SY * 4.50/6], target_max=[SX * 6.13/8, SY * 4.95/6])
exit_zone = shp.rectangle(grid, 
                          target_min=[SX * 5.67/8, SY * 5.47/6], 
                          target_max=[SX * 6.13/8, SY * 5.93/6])
entry_zone = shp.rectangle(grid, 
                           target_min=[SX * 1.50/8, SY * 1.87/6], 
                           target_max=[SX * 1.95/8, SY * 2.33/6])
parking_start = shp.rectangle(grid,
                              target_min=[SX * 2.30/8, SY * 1.87/6],
                              target_max=[SX * 2.75/8, SY * 2.33/6])
print('Done!')

In [ ]:
parking_lot = shp.union(parking_lot, 
                        shp.rectangle(grid, 
                                      target_min=[SX * 1.47/8, SY * 1.30/6], 
                                      target_max=[SX * 1.93/8, SY * 3.00/6]), # extended entry
                        shp.rectangle(grid,
                                      target_min=[SX * 1.47/8, SY * 1.85/6, +np.pi - np.pi/3], 
                                      target_max=[SX * 2.20/8, SY * 2.40/6, -np.pi + np.pi/5]), # box left direction
                        shp.rectangle(grid, 
                                      target_min=[SX * 1.20/8, SY * 1.20/6, -np.pi/5],
                                      target_max=[SX * 1.93/8, SY * 1.90/6, +np.pi/3]), # box right direction
)

In [ ]:
plot_map(
    # ('Grays', shp.project_onto(parking_lot, 0, 1)),
    ('Oranges', shp.project_onto(shp.union(kyrkogatan_left, kyrkogatan_right), 0, 1)),
    ('Purples', shp.project_onto(shp.union(nygatan_down, nygatan_up), 0, 1)),
    ('Greens', shp.project_onto(parking_lot, 0, 1)),
    alpha=0.6,
)

### Implementation

In [ ]:
from time import time
import numpy.lib.stride_tricks as st

AVOID_MARGIN = 0.1

solver_settings = hj.SolverSettings.with_accuracy("low")

def frs(times, target, constraints=None, *, mode='avoid'):
    times = np.asarray(times)
    dynamics = reach_dynamics if mode == 'reach' else avoid_dynamics
    values = hj.solve(solver_settings, dynamics, grid,
                      times, target, constraints)
    values = np.asarray(values)
    return values

def brs(times, target, constraints=None, *, mode='reach'):
    dynamics = reach_dynamics if mode == 'reach' else avoid_dynamics
    times = -np.asarray(times)
    if not  shp.is_invariant(grid, times, target):
        target = np.flip(target, axis=0)
    if not shp.is_invariant(grid, times, constraints):
        constraints = np.flip(constraints, axis=0)
    values = hj.solve(solver_settings, dynamics, grid,
                      times, target, constraints)
    values = np.asarray(values)
    values = np.flip(values, axis=0)
    return values

def find_windows(mask, N=1, M=None):
    """Find the indices of windows where at least N but less than M consequtive elements are true."""
    mask = np.asarray(mask)
    assert N <= len(mask)
    window_view = st.sliding_window_view(mask, window_shape=N)
    ix, = np.where(N == np.sum(window_view, axis=1))
    if M is not None:
        assert M <= len(mask)
        assert N < M
        window_view = st.sliding_window_view(mask, window_shape=M)
        jx = ix[ix + M <= len(mask)]
        iix, = np.where(np.sum(window_view[jx], axis=1) < M)
        ix = ix[iix]
    return ix

def earliest_window(mask, N=1, M=None):
    """Find the first window where at least N but less than M consequtive elements are true."""
    mask = np.asarray(mask)
    windows = find_windows(mask, N, M)
    if len(windows) == 0:
        return np.array([], int)
    i = windows[0] # Earliest window
    mask = mask[i:] if M is None else mask[i:i+M]
    for j, n in enumerate(mask.cumsum()[N-1:]):
        if n != j + N:
            break
    return np.arange(i, i+n)

def run(*passes, **kwargs):
    ALL_PASSES = ['pass1', 'pass2', 'pass3', 'pass4']
    passes = passes or ALL_PASSES
    if 'all' in passes:
        passes = ALL_PASSES
    passes = [name for name in ALL_PASSES if name in passes]

    returns = list(passes)

    for name in passes:
        assert name in ALL_PASSES, f'Invalid pass: {name}'
        i = ALL_PASSES.index(name)
        
        if i > 0 and ALL_PASSES[i-1] not in passes + list(kwargs):
            passes += [ALL_PASSES[i-1]]
    passes = [name for name in ALL_PASSES if name in passes]

    msg = 'Running the following passes: '
    msg += ', '.join(passes) + '\n'
    msg += '-' * (len(msg)-1)
    print(msg)

    def to_shared(values, **kwargs):
        return shp.project_onto(values, 0, 1, 2, **kwargs)
    
    def when_overlapping(a, b):
        return np.where(shp.project_onto(shp.intersection(a, b), 0) <= 0)[0]
    
    def pass1():
        print('\n', 'Pass 1: Initial BRS')
        times = kwargs['times']
        rules = kwargs['rules']
        end = kwargs['end']

        target = shp.make_tube(times, end)
        constraints = rules

        start_time = time()
        result = brs(times, target, constraints)
        stop_time = time()

        print(f'Time To Compute: {stop_time - start_time:.02f}')
        return result
    
    def pass2():
        print('\n', 'Pass 2: Avoidance')
        times = kwargs['times']
        avoid = kwargs['avoid']
        end = kwargs['end']

        if not avoid:
            result = kwargs['pass1']
        else:
            result = kwargs['pass1'].copy()

            avoid_target = shp.union(
                *map(lambda target: to_shared(target, keepdims=True), avoid),
            ) - AVOID_MARGIN # increase avoid set with heuristic margin

            interact_window = when_overlapping(result, avoid_target)
            if 0 < interact_window.size:
                i, j = interact_window[0], interact_window[-1] + 1

                # Recompute solution until after last interaction
                constraints = shp.setminus(result, avoid_target)[:j+1]
                target = result[:j+1]    # last step is target to reach pass1
                target[:-1] = end       # all other steps are recomputed to end
                
                start_time = time()
                result[:j+1] = brs(times[:j+1], target, constraints)
                stop_time = time()
            
                print(f'Time To Compute: {stop_time - start_time:.02f}')
                print(f'First Interaction: {times[i]:.01f}')
                print(f'Last Interaction: {times[j-1]:.01f}')
        return result

    def pass3():
        print('\n', 'Pass 3: Planning, Departure')
        result = kwargs['pass2'].copy()
        times = kwargs['times']
        start = kwargs['start']

        depart_target = shp.intersection(result, start)
        depart_window = earliest_window(shp.project_onto(depart_target, 0) <= 0, 5)
        assert depart_window.size > 0, 'No window to depart'
        i = depart_window[0] # Earliest departure
        j = depart_window[min(11, len(depart_window)-1)] # Depart within 2 seconds of earliest departure
        depart_target[j:] = 1

        start_time = time()
        result[i:] = frs(times[i:], depart_target[i:], result[i:])
        result[:i] = 1 # Remove all values before departure
        stop_time = time()

        print(f'Time To Compute: {stop_time - start_time:.02f}')
        print(f'Earliest Departure: {times[i]:.01f}')
        print(f'Latest Departure: {times[j-1]:.01f}')
        return result

    def pass4():
        print('\n', 'Pass 4: Planning, Arrival')
        result = kwargs['pass3'].copy()
        times = kwargs['times']
        start = kwargs['start']
        end = kwargs['end']

        depart_target = shp.intersection(result, start)
        depart_window = earliest_window(shp.project_onto(depart_target, 0) <= 0, 5)
        assert depart_window.size > 0, 'No window to depart'
        i = depart_window[0] # Earliest departure
        j = depart_window[min(11, len(depart_window)-1)] # Depart within 2 seconds of earliest departure
        depart_target[j:] = 1

        arrival_target = shp.intersection(result, end)
        arrival_window = earliest_window(shp.project_onto(arrival_target, 0) <= 0, N=5)
        assert arrival_window.size > 0, 'No window to arrive'
        m = arrival_window[0] # Earliest arrival
        n = arrival_window[min(11, len(depart_window)-1)] # Arrive within 2 seconds of earliest arrival
        arrival_target[n:] = 1

        start_time = time()
        result[i:n] = brs(times[i:n], arrival_target[i:n], result[i:n])
        result[n:] = 1 # Remove all values after arrival
        stop_time = time()

        print(f'Time To Compute: {stop_time - start_time:.02f}')
        print(f'Earliest Arrival: {times[m]:.01f}')
        print(f'Latest Arrival: {times[n-1]:.01f}')
        return result

    if 'pass1' in passes:
        kwargs['pass1'] = pass1()
    if 'pass2' in passes:
        kwargs['pass2'] = pass2()
        if 'pass1' not in returns: del kwargs['pass1']
    if 'pass3' in passes:
        kwargs['pass3'] = pass3()
        if 'pass2' not in returns: del kwargs['pass2']
    if 'pass4' in passes:
        kwargs['pass4'] = pass4()
        if 'pass3' not in returns: del kwargs['pass3']
    
    return [kwargs[name] for name in returns]

def run_many(*objectives, **kwargs):
    results = []
    for o in objectives:
        results += run('pass4', **o, **kwargs, avoid=results)
        print('\n')
    return results

In [ ]:
if False:
    (v1_pass1,) = run('pass1', # 'pass3',  # 'pass4',
        rules=shp.union(kyrkogatan_right, nygatan_up),
        avoid=[],
        start=entry_zone,
        end=exit_zone,
        times=new_timeline(20),
    )
else:
    (v2_pass1,) = run('pass1',
        rules=shp.union(kyrkogatan_right, kyrkogatan_left, parking_lot),
        avoid=[],
        start=parking_start,
        end=shp.union(parking_spot_1, parking_spot_2),
        times=new_timeline(20),
    )

In [ ]:
plot_map(
    # ('Grays', shp.project_onto(shp.union(
    #         shp.union(kyrkogatan_left, parking_lot),
    #         shp.union(parking_spot_entry_1, parking_spot_1),
    #         shp.union(parking_spot_entry_2, parking_spot_2),
    #     ), 0, 1)
    # ),

    ## Parking ##
    ('Greys', shp.project_onto(v1_pass1, 1, 2)),
    ('Blues', shp.project_onto(v2_pass1, 1, 2)),
    # ('Greens', shp.project_onto(parking_start, 0, 1)),
    ('Greens', shp.project_onto(shp.union(parking_spot_1, parking_spot_2), 0, 1)),
    
    # ## Environment ##
    # ('Oranges', shp.project_onto(shp.union(kyrkogatan_left, kyrkogatan_right), 0, 1)),
    # ('Purples', shp.project_onto(shp.union(nygatan_down, nygatan_up), 0, 1)),
    # ('Greens', shp.project_onto(parking_lot, 0, 1)),
    
    # alpha=0.7,
)

In [ ]:
I, F = interact_tubes(
    new_timeline(10),
    # ('Greys', shp.make_tube(new_timeline(10), kyrkogatan_left)),
    ('Blues',   v1_pass1),
    # ('Greens',  v1_pass3),
    # ('Reds',    v1_pass4),
    eye=EYE_SSW,
)
if False:
    r = I(F)
else:
    r = F().show(config=dict(toImageButtonOptions=dict(height=720, width=720, scale=6)))

r

In [ ]:
vf = v1_pass1
vf = shp.project_onto(vf, 0, 5)

idx = np.where(vf <= 0)

itov = lambda i: grid.coordinate_vectors[4][i]
xs = new_timeline(8)
miny = np.array([itov(min(idx[1][idx[0] == i])) for i in range(len(xs)) if len(idx[1][idx[0] == i])])
maxy = np.array([itov(max(idx[1][idx[0] == i])) for i in range(len(xs)) if len(idx[1][idx[0] == i])])
xs = np.array([xs[i]for i in range(len(xs)) if len(idx[1][idx[0] == i])])
print(xs)
plt.figure()
plt.fill_between(xs, miny, maxy)
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.title('Velocity Profile')